In [1]:
import os
import cv2
from torchvision import transforms
from PIL import Image
import torch
import numpy as np
import time
import subprocess
import json
import pandas as pd

In [2]:
from basicsr.archs.edvr_arch import EDVR
from basicsr.archs.edvrOriginal_arch import EDVRoriginal

In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [4]:
to_tensor = transforms.ToTensor()
toPIL = transforms.ToPILImage()

In [5]:
video_540='video_540.mp4'
video_360='video_360.mp4'
video_216='video_216.mp4'
video_4k='video_4k.mp4'

In [6]:
video_list=[video_540,video_360,video_216]
resolution=['540p','360p','216p']

In [7]:
EDVR_x4=EDVRoriginal(num_feat=128,num_reconstruct_block=40)
EDVR_x6=EDVR(num_feat=128,num_reconstruct_block=40,upscale=6,scale_1=2,scale_2=3)
EDVR_x10=EDVR(num_feat=64,num_reconstruct_block=40,upscale=10,scale_1=2,scale_2=5)

In [8]:
state_x4=torch.load('./models/EDVR_L_x4_SR_REDS_official-9f5f5039.pth')
state_x6=torch.load('./models/EDVR_x6.pth')
state_x10=torch.load('./models/EDVR_x10.pth')

In [9]:
EDVR_x4.load_state_dict(state_x4['params'], strict=False)
EDVR_x6.load_state_dict(state_x6['params'], strict=False)
EDVR_x10.load_state_dict(state_x10['params'], strict=False)

<All keys matched successfully>

In [10]:
model_list=[EDVR_x4,EDVR_x6,EDVR_x10]
model_list=[EDVR_x4,EDVR_x6,EDVR_x4]

In [11]:
def SR_upscale(model,in_img,num_frame=5):
    in_img=in_img[:,:,::-1]
    img_tensor=torch.unsqueeze(to_tensor(in_img.copy()),0)
    img_tensors=torch.stack([img_tensor]*num_frame,1)
    with torch.no_grad():
        model.eval()
        torch.cuda.empty_cache()
        model=model.to(device)
        img_tensors=img_tensors.to(device)
        pred=model(img_tensors)
    torch.cuda.empty_cache()
    return np.array(toPIL(pred.data[0]).convert('RGB'))

In [12]:
def upscale(video,sr_freq,resolution,model,write_video=False,output=None,ref_video=None):
    count=0
    cap=cv2.VideoCapture(video)
    video_writer=None
    if write_video:
        fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
        video_writer=cv2.VideoWriter(output,fourcc,60,(3840,2160),True)
    start=time.time()
    t2=0
    while True:
        s=time.time()
        ret,frame=cap.read()
        if not ret:
            break
        if count%sr_freq==0:
            frame=SR_upscale(model,frame)
            # if the result is not good enough on x6 and x10, it can try first scale 4 times, then use BIcubic to upscale to 4K video
            #if resolution !='540p':
            #    frame=cv2.resize(frame,(3840,2160),interpolation=cv2.INTER_CUBIC)
        else#:
            frame=cv2.resize(frame,(3840,2160),interpolation=cv2.INTER_CUBIC)
        frame=np.uint8(frame)
        e=time.time()
        t2+=(e-s)
        if write_video:
            video_writer.write(frame)
        count+=1
    end=time.time()
    total=end-start
    fps=count/total
    results=dict()
    results['sr_freq']=sr_freq
    results['resolution']=resolution
    results['total_time']=total
    results['fps']=fps
    if write_video:
        results['total_time_only_upscale']=t2
        results['fps_only_upscale']=count/t2
        metrics=subprocess.run(['ffmpeg-quality-metrics',output,ref_video,'--metrics','ssim', 'vmaf', 'psnr','-r','60','-t','12'],capture_output=True)
        metrics_result=json.loads(metrics.stdout.decode('utf-8'))['global']
        results['psnr']=metrics_result['psnr']['psnr_avg']['average']
        results['ssim']=metrics_result['ssim']['ssim_avg']['average']
        results['vmaf']=metrics_result['vmaf']['vmaf']['average']   
        video_writer.release()
    return results

In [13]:
df_result=pd.DataFrame(columns=['sr_freq','resolution','total_time','fps','total_time_only_upscale','fps_only_upscale','psnr','ssim','vmaf'])

In [15]:
result_video_path='./SR_result_video'
it=1
for i,video in enumerate(video_list):
    
    model=model_list[i]
    #model=EDVR_x4
    for sr_freq in range(10,31):
        print('doing',it)
        out_video=os.path.join(result_video_path,f'r_{resolution[i]}_{sr_freq}_001.avi')
        result=upscale(video,sr_freq,resolution[i],model,write_video=True,output=out_video,ref_video=video_4k)
        df_result=df_result.append(result,ignore_index=True)
        df_result.to_csv('result_sr3.csv',index=False)
        print('finish',it)
        it+=1
    

doing 1
finish 1
doing 2
finish 2
doing 3
finish 3
doing 4
finish 4
doing 5
finish 5
doing 6
finish 6
doing 7
finish 7
doing 8
finish 8
doing 9
finish 9
doing 10
finish 10
doing 11
finish 11
doing 12
finish 12
doing 13
finish 13
doing 14
finish 14
doing 15
finish 15
doing 16
finish 16
doing 17
finish 17
doing 18
finish 18
doing 19
finish 19
doing 20
finish 20
doing 21
finish 21
doing 22
finish 22
doing 23
finish 23
doing 24
finish 24
doing 25
finish 25
doing 26
finish 26
doing 27
finish 27
doing 28
finish 28
doing 29
finish 29
doing 30
finish 30
doing 31
finish 31
doing 32
finish 32
doing 33
finish 33
doing 34
finish 34
doing 35
finish 35
doing 36
finish 36
doing 37
finish 37
doing 38
finish 38
doing 39
finish 39
doing 40
finish 40
doing 41
finish 41
doing 42
finish 42
doing 43
finish 43
doing 44
finish 44
doing 45
finish 45
doing 46
finish 46
doing 47
finish 47
doing 48
finish 48
doing 49
finish 49
doing 50
finish 50
doing 51
finish 51
doing 52
finish 52
doing 53
finish 53
doing 54
fi

RuntimeError: CUDA out of memory. Tried to allocate 204.00 MiB (GPU 3; 39.59 GiB total capacity; 690.96 MiB already allocated; 59.19 MiB free; 708.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF